In [1]:
import networkx as nx
import numpy as np

In [33]:
def propagate_neighbours(graph:nx.Graph, new_active, p, seed=None):
    neighbors = []
    for node in new_active:
        neighbors += g.neighbors(node)
    
    if seed:
        np.random.seed(seed)

    activation = np.random.uniform(low=0, high=1, size=len(neighbors)) < p
    activated_neighbors = list(np.extract(activation, sorted(neighbors)))

    return activated_neighbors

def independent_cascade_model(graph:nx.Graph, activated_nodes, p=.2, seed=None):
    new_active = activated_nodes[:]
    list_active = activated_nodes[:]
    while new_active:
        activated_neighbors = propagate_neighbours(graph, new_active, p=p, seed=seed)
        new_active = list(set(activated_neighbors) - set(list_active))
        list_active  += new_active
    
    spread = len(list_active)

    return {'spread':spread, 'list_active':list_active}

def n_independent_cascade_model(graph:nx.Graph, activated_nodes, n, p=.2, seed_list=None):
    if seed_list is None:
        seed_list = [None]*n
    spread_sum = 0
    for seed in seed_list:
        icm_dict = independent_cascade_model(g, activated_nodes, p=p, seed=seed)
        spread_sum += icm_dict.get('spread')
    return spread_sum

def greedy_influence_maximisation(g:nx.Graph, k, n, p=.2, seed=None):
    A0 = []
    if seed:
        np.random.seed(seed)
    g_seed = np.random.randint(np.iinfo(np.int32).max ,size=n)
    for node_idx in range(k):
        nodes = set(g.nodes()) - set(A0)
        spread_score = {node:n_independent_cascade_model(g, A0 + [node], n, p=p, seed_list=g_seed) for node in nodes}
        max_node = -1; max_spread = -1
        for n, total_spread in spread_score.items():
            if total_spread > max_spread:
                max_node = n
                max_spread = total_spread
        A0.append(max_node)
    # list_active = spread_score.get(A0[-1])['list_active']
    return {'A0':A0, 
           # 'spread':len(list_active), 
           # 'list_active':list_active
           }


In [48]:
g = nx.karate_club_graph()

info_icm = independent_cascade_model(g, [0], p=.1)
print(info_icm)

info= greedy_influence_maximisation(g, 3, n=100, seed=None)
print(info)

{'spread': 2, 'list_active': [0, 12]}
{'A0': [33, 0, 1]}
